### Application of functions looped over a profile
This is needed in order to: 
- [x] make a row/column indexation of grid cells
- [x] enable applying functions over a row or column
- [x] apply the Lacey functions on each profile
We focus on one single profile application first.

In [ ]:
import orprofile
import geopandas as gpd
import os
import xugrid as xu
import xarray as xr
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
fn_splines = "../examples/data/splines.geojson"
fn_points = "../examples/data/bamboi_survey.geojson"
splines = gpd.read_file(fn_splines)

mesh = orprofile.api.mesh.Mesh(splines, n=10, m=10, points=fn_points)
mesh.mesh2d.plot()


In [ ]:
mesh.mesh2d

### organizing rows and columns into a XUGridDataSet

In [ ]:
# make a columns and row coordinate for each node
columns = np.repeat([np.arange(mesh.n)], mesh.m, axis=0).flatten()
rows = np.repeat(np.arange(mesh.m), mesh.n)

# prepare UgridDataArrays, using the grid
da_rows = xu.UgridDataArray(
    xr.DataArray(
        data=rows,
        dims=[mesh.mesh2d.face_dimension]
    ),
    grid=mesh.mesh2d,
)
da_cols = xu.UgridDataArray(
    xr.DataArray(
        data=columns,
        dims=[mesh.mesh2d.face_dimension]
    ),
    grid=mesh.mesh2d,
)

# combine everything into a UgridDataSet
ds = xu.UgridDataset(grids=mesh.mesh2d)
ds.coords["rows"] = da_rows
ds.coords["cols"] = da_cols
# ds.coords["node_x"] = (mesh.mesh2d.face_dimension, mesh.mesh2d.node_x)
# ds.coords["node_y"] = (mesh.mesh2d.face_dimension, mesh.mesh2d.node_y)
ds["rows"] = da_rows
ds["cols"] = da_cols

# make a fancy plot
f, axs = plt.subplots(ncols=2, figsize=(13, 4))
ds.cols.ugrid.plot(ax=axs[0])
ds.rows.ugrid.plot(ax=axs[1])
axs[0].set_title("column indexes")
axs[1].set_title("row indexes")
axs[0].grid(linestyle="--")
axs[1].grid(linestyle="--")


In [ ]:
ds_g = ds.groupby("rows")
ds_1 = list(ds_g)[0][1]
grid_sel = mesh.mesh2d.isel(mesh2d_nFaces=ds_1.mesh2d_nFaces.values)
ds_1_ug = xu.UgridDataset(ds_1, grids=grid_sel)
ds_1_ug.ugrid.grid.node_x
# ds_1_ug.ugrid.set_node_coords



In [ ]:
list(ds_g)[1][1]

### develop a mapping function that can be applied per row or column
We need a special mapping function because xugrid cannot yet handle groupby / map combinations without
destroying the ugrid property. Here we develop functions that add the grid for each group internally until
xugrid is updated.

In [ ]:
# general mapping function, which must become an internal method in mesh
def map_func(row, f, ugrid, name="new", **kwargs):
    grid_sel = ugrid.isel(mesh2d_nFaces=row.mesh2d_nFaces.values)
    row_ug = xu.UgridDataset(row, grids=grid_sel)
    # now apply the function
    result = f(row_ug, **kwargs)
    result.name = name
    return result

# specific function that computes something. This can be made by the user and applied
def get_dist(row):
    # get the centroid coordinates of each face
    x = row.ugrid.to_geodataframe().centroid.x
    y = row.ugrid.to_geodataframe().centroid.y
    # compute the difference in distance per grid cell from one bank to the other
    ds = (np.diff(x)**2 + np.diff(y)**2)**0.5
    # distance from bank
    s = np.cumsum(np.pad(ds, (1, 0), "constant"))
    # use the "cols" variable as a template
    da = row["cols"]
    da[:] = s
    return xr.DataArray(da)




In [ ]:
distance = xu.UgridDataArray(
    ds_g.map(
        map_func,
        f=get_dist,
        name="distance from bank",
        ugrid=mesh.mesh2d
    ),
    grid=mesh.mesh2d
)
ax = plt.axes()
distance.ugrid.plot(ax=ax)
ax.axis("equal")


### functionality migrated to orprofile package
Below we also test the functionality within tre orprofile package in `Mesh.api.mesh`, this should give the same results

In [ ]:
distance = mesh.map_rowwise(orprofile.profile.get_dist, name="distance from bank")
ax = plt.axes()
distance.ugrid.plot(ax=ax)
ax.axis("equal")
